# 2つのガウス分布を含む混合ガウス分布のためのEMアルゴリズム

千葉工業大学 上田 隆一

(c) 2017 Ryuichi Ueda

This software is released under the MIT License, see LICENSE.

## はじめに

このコードは、2つの2次元ガウス分布を含む混合ガウス分布から生成されたデータについて、EMアルゴリズムでパラメータを求めるためのEMアルゴリズムの実装例です。

## アルゴリズムを適用される対象になるデータの生成

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as pylab

# グループ1のガウス分布
mu1_ans = [170,70]
cov1_ans = [[6**2,-30],[-30,8**2]]
X1 = np.random.multivariate_normal(mu1_ans,cov1_ans,100).T #サンプル生成

# グループ2のガウス分布
mu2_ans = [130,50]
cov2_ans = [[5.**2,20],[20,6**2]]
X2 = np.random.multivariate_normal(mu2_ans,cov2_ans,50).T #サンプル生成

# 2つのグループを混ぜる
X = np.column_stack((X1,X2))

# 描画
pylab.scatter(X[0],X[1],color='g',marker='+')